# Introduction to Bash shell scripting

A Bash shell script is a plain-text file containing a sequence of commands intended to be run by a Bash shell. When
a script is run, the shell reads the file and interprets the commands contained in the script; the shell is both
a command line interface that the user can interact with directly and an interpreter for scripts.

The script file must be readable and executable, and the operating system needs to know where the script is located.
If the script is not in a directory specified by the current path then the path to the script must be specified
by the caller of the script.

Being plain-text files, a script has a filename. There are no universally accepted style rules for naming scripts
but we will use the following conventions for script filenames:

* ends with the extension `.sh`
* starts with a lowercase letter
* contains only alphanumeric characters and the underscore
* multiword script names use the underscore to separate words


## Some simple Bash scripts

<div class="alert alert-block alert-info">
    You can find all of the scripts in this notebook in the subdirectory containing this notebook:
    <code>./scripts/introduction_to_bash_scripting</code>
</div>


The following script (named `hello.sh`) is one possible implementation of the traditional 'Hello, world' program:

In [ ]:
#!/bin/bash

echo 'Hello, world!'

Jupyter notebook can run the script within the Jupyter notebook environment. Running the above cell will
produce the output `Hello, world!`.

Of course, you would normally write scripts outside of the Jupyter environment. To do so, you might
do the following:

1. start a text editor program
2. type the lines of the script into the editor
3. save the file with the filename `hello.sh`
4. open a terminal and changing to the directory containing the saved `hello.sh` file
5. change the permissions of `hello.sh` so that it is readable and executable by the current user
6. run the script by typing `./hello.sh` and pressing enter
    * if the directory that `hello.sh` is on the current path then the script can be run by typing `hello.sh`
    
All of the scripts found in this notebook are located in a subdirectory named
`./scripts/introduction_to_bash_scripting`; thus, we can run the script by running a Jupyter cell that
calls the script directly:

In [ ]:
./scripts/introduction_to_bash_scripting/hello.sh

Alternatively, we can run a Jupyter cell that changes to the script directory and calls the script:

---

```sh
cd ./scripts/introduction_to_bash_scripting
./hello.sh
```

---

We will avoid changing directories in this notebook because doing so affects the current working
directory for all cells in this notebook.

The first line of the script should start with the characters `#!` (called a 
[*hash-bang* or *shebang*](https://en.wikipedia.org/wiki/Shebang_(Unix))). When a text file starting with a
hash-bang is used as an executable program in a Unix-like environment, the remainder of the first line of the
script is treated as an interpreter directive. The interpreter directive specifies the program that should
be used to interpret the commands in the script; thus:

```sh
#!/bin/bash
```

specifies that the program `/bin/bash` should be used to interpret the script commands. On Linux systems, `/bin/bash`
is the Bash shell program.

It is possible to run a script that does not have execute permission by running a shell program an passing
the script pathname as an argument. For example, typing the following into a terminal:

```sh
bash hello.sh
```

will run the `hello.sh` script (assuming that the current working directory contains the script).

Any command that can be entered at the command line can appear in a script. For example, the following
script  (named `fortunecow.sh`) uses a pipe to send the output of the `fortune` program to the input of the
`cowsay` command:

In [ ]:
#!/bin/bash

fortune | cowsay -f moose

An example of a script that actually does something constructive is the following script  (named `makesh.sh`)
that creates a
skeleton script file named `tmp.sh` in the current directory and sets the execute permission of the file for the
current user:

In [ ]:
#!/bin/bash

echo '#!/bin/bash' > tmp.sh
chmod u+x tmp.sh

If you run the above cell, you will see no output which follows the [Rule of Silence](./rules_of_the_unix_philosophy.ipynb). The script creates the file `tmp.sh` in the current working directory.
We can verify that `tmp.sh` has been created by listing the contents of the current directory in file modification
order (most recently modified files first):

In [ ]:
ls -t

## Comments

Bash supports line ending comments (similar to Python's `#` comment and Java's `//` comment) where the
text of the comment is ignored by the shell. The comment delimiter is the `#` character:

In [ ]:
#!/bin/bash

# comment.sh

# This program does nothing (almost)

Running the above cell seemingly does nothing but every command (not just scripts) in a UNIX enviroment sets a
value called the *exit status* when it finishes running. The exit status of the most recently completed
command is stored in the special parameter `?`.

<div class="alert alert-block alert-info">
    <p>
    Bash defines a <it>parameter</it> as being an entity that stores a value. A <it>variable</it>
    is a parameter that has a name starting with a letter. Bash defines what are called
    <it>special parameters</it> that are represented using symbols and
    <it>positional parameters</it> that are represented using integer values.
    </p>
    <ul>
        <li><a href="https://www.gnu.org/software/bash/manual/html_node/Special-Parameters.html">special
            parameters documentation</a></li>
        <li><a href="https://www.gnu.org/software/bash/manual/html_node/Positional-Parameters.html">positional
            parameters documentation</a></li>
    </ul>
    <p>
        The programmer cannot assign a value to a special or positional parameter using the assignment
        operator `=`; such parameters have values that are normally set by the shell.
    </p>
</div>


To obtain the value stored in a parameter, we use the `$` character in front of the parameter name.
To print the value of the exit status of the `comment.sh` script we obtain the value of `?` using
the notation `$?` and `echo` the value:

In [ ]:
./scripts/introduction_to_bash_scripting/comment.sh
echo $?

A successful command should produce an exit status of `0` and an unsuccessful command should produce a non-zero
integer value between `1` and `255`. We will learn more about setting the exit status of a script in other
notebooks.

Comments may appear after a command:

```sh
#!/bin/bash

echo '#!/bin/bash' > tmp.sh    # might be useful if the user could choose the filename
chmod u+x tmp.sh
```

A quoted or escaped `#` does not delimit a comment:

In [ ]:
#!/bin/bash

# notcomment.sh

echo '# This is not a comment'

A comment always terminates a line; thus, it is not possible to insert a comment in the middle of a line. The intent
of the following script is to list all of the filenames ending with `.txt`:

In [ ]:
#!/bin/bash

# badcomment.sh

ls #list_all_.txt_files .txt

Instead, the script lists all of the (non-hidden) files in the current directory because:

```sh
ls #list_all_.txt_files .txt
```

is interpreted by the shell as:

```sh
ls
```

## Command line arguments

As we have seen, most commands accept or require external information in the form of command line arguments.
Shell scripts can also make use of command line arguments. The shell provides what are called
*positional parameters* that contain the strings on the command line.

The number of command line arguments used to run a script are stored in the special shell parameter named `#`.

The individual command line arguments are stored in the special shell parameters named `0`, `1`, `2`, and so on.

The following script prints the number of command line arguments used to run a script and the values of those
arguments:

---

```sh
#!/bin/bash

# viewargs.sh: view command line arguments

echo "
Number of arguments: $#
\$0 = $0
\$1 = $1
\$2 = $2
\$3 = $3
\$4 = $4
\$5 = $5
\$6 = $6
\$7 = $7
\$8 = $8
\$9 = $9
\${10} = ${10}
"
```

---

Note that there is no way to pass command line arguments to a script defined in a Jupyter notebook cell
so all scripts that allow for command line arguments will be shown inline instead of as a runnable cell.

The following cell runs `viewargs.sh` with zero command line arguments:

In [ ]:
./scripts/introduction_to_bash_scripting/viewargs.sh

Notice that it is *not* considered an error to attempt to obtain the value of a parameter that is not set.
Calling `viewargs.sh` with no command line arguments implies that the positional parameters `1`, `2`, and so
on are not set but we can still attempt to obtain their values using `$1`, `$2`, and so on, without an
error being raised by Bash.

Readers are encouraged to run the `viewargs.sh` script by modifying the above cell, or in a terminal and passing
in various command line arguments to view the values stored in the positional parameters. Doing so should quickly lead you to conclude that:

* `$#` is the number of command line arguments passed to the script
    * e.g., `./viewargs.sh` has 0 command line arguments, `./viewargs.sh abc` has 1 command line argument, `/viewargs.sh abc 123` has 2 command line arguments, and so on
* `$0` is the string equal to the pathname used to call the script
* `$1` is the string equal to the first command line argument used to the call the script
* `$2` is the string equal to the second command line argument used to the call the script, and so on

It is always legal to enclose a parameter inside of braces when using `$` to obtain its value, but
for positional parameters greater than or equal to `10` you must enclose the parameter inside
of braces `{}`. 

We can easily modify our existing scripts to modify their behaviour using command line arguments. For example, we
can modify `hello.sh` to greet the user:

---

```sh
#!/bin/bash

# hellou.sh

echo "Hello, ${1}!"
```

---

The following cell runs `hellou.sh` with no command line arguments:

In [ ]:
./scripts/introduction_to_bash_scripting/hellou.sh

Readers are encouraged to run the `hellou.sh` script by modifying the above cell, or in a terminal and passing
in various command line arguments to view the output of the script.

Similarly, we can modify `makesh.sh` to accept the name of the file to create as a command line argument:

---

```sh
#!/bin/bash

# namedsh.sh

echo '#!/bin/bash' > ${1}
chmod u+x ${1}
```

---

The following cell runs `namedsh.sh` with one command line argument and then lists the directory
contents:

In [ ]:
./scripts/introduction_to_bash_scripting/namedsh.sh myscript.sh
ls